In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv("")

In [4]:
df = pd.read_csv("catalog_datasets.csv", dtype=str, sep=";")

In [8]:
df["metric.views"] = df["metric.views"].astype(float)

In [10]:
df = df.sort_values(by="metric.views", ascending=False)

In [12]:
dff = df[:1000]

In [15]:
dff["quality_score"] = df["quality_score"].astype(float)

/var/folders/cg/b2_z5vjn52x7nyrtvtmwqsqh0000gp/T/ipykernel_14710/3378737279.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff["quality_score"] = df["quality_score"].astype(float)


In [16]:
dff["quality_score"].mean()

0.69062

In [17]:
df = pd.read_csv("catalog_discussions.csv", dtype=str, sep=";")

In [19]:
df.columns

Index(['id', 'user', 'subject', 'title', 'size', 'messages', 'created',
       'closed', 'closed_by'],
      dtype='object')

In [24]:
df["user"].value_counts()

Kisio Digital                289
Frédéric ALLARI              214
DELETED DELETED              199
Karima Mekkaoui              172
Christopher Do Nascimento    139
                            ... 
thierry pastre                 1
Mariona Segú                   1
Abdourahmane KHOUMA            1
Nicolas Mougamadou             1
Frédéric Hars                  1
Name: user, Length: 7308, dtype: int64

In [25]:
df

,id,user,subject,title,size,messages,created,closed,closed_by
0,6460fa1fd3e6eb3e2998734a,Christophe SAHUC,Tuto : accéder au débit des fleuves français v...,Collecte de débit à partir d’une station hydro,1,"Bonjour,\nJe suis à la recherche d’un programm...",2023-05-14T17:11:27.131000,NaN,NaN
1,645f874d7758e57434f78a60,Philippe MAITRALLAIN,Base d'Adresses Nationale Ouverte - BANO,erreur dans l'adresse,2,"Bonjour,\nSur la commune de Chevannes dans l'E...",2023-05-13T14:49:17.097000,NaN,NaN
2,645ea42804632eafc1b221f6,Clément Romanowski,Offre de transport du réseau de Vannes - Kicéo...,Problème dans calendar.txt,2,"Bonjour,\r\nLe fichier calendar.txt, qui est c...",2023-05-12T22:40:08.241000,NaN,NaN
3,645e7fe0fe8893510cdf1f42,Kevin Lo,Base Sirene des entreprises et de leurs établi...,Will the data be removed or stay forever?,1,"If a establishment or legal unit is closed, wi...",2023-05-12T20:05:20.702000,NaN,NaN
4,645e305cd57cf2918391a81d,Cyril LEDRU,Résultats du contrôle sanitaire de l'eau distr...,"open data sur les contrôles sanitaires ""piscin...",1,"Bonjour,\nJe souhaiterai utiliser les données ...",2023-05-12T14:26:04.215000,NaN,NaN
...,...,...,...,...,...,...,...,...,...
12076,5566b28754b314fff698bbcb,Benoit DAVID,Relief Naturel - données géographiques,site inaccessible,1,site inaccessible,2013-12-19T10:37:45.197000,NaN,NaN
12077,5566b28754b314fff698bbca,Jean-Christophe MARTI,Base nationale de Gestion ASsistée des Procédu...,Le fichier est annoncé comme un CSV mais je n'...,1,Le fichier est annoncé comme un CSV mais je n'...,2013-12-18T22:40:12.111000,NaN,NaN
12078,5566b28754b314fff698bbc9,François BELLIN,Plan local de l'urbanisme - PLU,Le lien pour télécharger les données ne foncti...,1,Le lien pour télécharger les données ne foncti...,2013-12-18T22:26:47.529000,NaN,NaN
12079,5566b28754b314fff698bbc8,Frédéric Hars,La comptabilité nationale,"Erreur d'URL sur le document ""Principaux indic...",1,"Erreur d'URL sur le document ""Principaux indic...",2013-12-18T18:17:31.356000,NaN,NaN


In [3]:
import request<<
s

SyntaxError: invalid syntax (2390534649.py, line 1)

In [4]:
from datetime import datetime
date_format = "%Y-%m-%dT%H:%M:%S" 
toto = datetime.strptime("2023-03-22T18:43:46.843000+00:00"[:19], date_format) - datetime.strptime("2023-03-21T09:13:12.843000+00:00"[:19], date_format)


In [5]:
toto.days + (toto.seconds/(3600*24))

1.396226851851852

In [6]:
34234/(3600*24)

0.39622685185185186

In [7]:
from datetime import date
from dateutil.relativedelta import relativedelta

oneyearago = date.today() - relativedelta(years=1)
oneyearago = oneyearago.strftime("%Y-%m-%d")

In [8]:
page = 1
nb_discussions = 0
nb_discussions_with_answer = 0
time_to_answer = []
actual_date = date.today().strftime("%Y-%m-%d")
while actual_date > oneyearago:
    print(page, actual_date)
    r = requests.get("https://www.data.gouv.fr/api/1/discussions/?page_size=100&page="+str(page))
    data = r.json()
    for item in data["data"]:
        actual_date = item["discussion"][0]["posted_on"]
        nb_discussions += 1
        if len(item["discussion"]) > 1:
            nb_discussions_with_answer += 1
            date_format = "%Y-%m-%dT%H:%M:%S" 
            first_date = datetime.strptime(item["discussion"][0]["posted_on"][:19], date_format)
            second_date = datetime.strptime(item["discussion"][1]["posted_on"][:19], date_format)
            ecart = second_date - first_date
            ecart_jour = ecart.days + (ecart.seconds/(3600*24))
            if ecart_jour > 30:
                time_to_answer.append(30)
            else:
                time_to_answer.append(ecart_jour)
        else:
            time_to_answer.append(30)
    page = page + 1

1 2023-09-11
2 2023-08-23T14:01:38.600000+00:00
3 2023-07-27T12:35:35.380000+00:00
4 2023-06-30T09:34:23.993000+00:00
5 2023-06-07T12:52:01.118000+00:00
6 2023-05-11T14:00:57.405000+00:00
7 2023-04-13T16:59:27.536000+00:00
8 2023-03-16T16:28:13.299000+00:00
9 2023-02-14T21:25:29.180000+00:00
10 2023-01-24T21:34:01.058000+00:00
11 2023-01-03T14:08:13.388000+00:00
12 2022-12-11T12:28:54.479000+00:00
13 2022-11-22T07:41:31.581000+00:00
14 2022-10-27T14:33:28.047000+00:00
15 2022-10-06T15:03:08.793000+00:00
16 2022-09-13T14:12:15.343000+00:00


In [9]:
nb_discussions

1600

In [10]:
nb_discussions_with_answer

1072

In [65]:
len(time_to_answer)

1600

In [68]:
import numpy as np

In [69]:
arr = np.array(time_to_answer)

In [70]:
arr.mean()

14.307382349537038

In [72]:
arr2 = []
for item in time_to_answer:
    if item != 30:
        arr2.append(item)

In [73]:
arr = np.array(arr2)

In [74]:
arr.mean()

4.379399754346182